In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

full = pd.read_csv('/content/drive/My Drive/Colab Notebooks/InputPythonUTF8.csv')

In [ ]:
full.columns = full.columns.str.replace(".", "-")

In [ ]:
oneday_races_cobbles = ['omloop-der-vlaamse-gewesten2'
  ,'paris-roubaix-juniors'
  ,'ronde-van-vlaanderen-juniores'
  ,'tour-des-flandres-espoirs'
  ,'paris-roubaix-espoirs'
  ,'kattekoers']

oneday_races_MJ = ['bernaudeau-junior'
  ,'gp-general-patton'
  ,'gp-dell-arno'
  ,'trofeo-buffoni'
  ,'trofeo-comune-di-vertova'
  ,'trofeo-emilio-paganessi'
  ,'le-trophee-centre-morbihan'
  ,'european-championship-mj'
  ,'uci-world-championships-mj']

oneday_races_hilly_U23 = ['liege-bastogne-liege-u23'
  ,'coppa-della-pace-trofeo-flli-anelli'
  ,'gp-palio-del-recioto'
  ,'gp-capodarco'
  ,'gran-premio-della-liberazione'
  ,'gp-industrie-del-marmo'
  ,'gp-di-poggiana'
  ,'piccolo-giro-di-lombardia'
  ,'ruota-d-oro-gp-festa-del-perdono'
  ,'gp-industria-e-commercio2'
  ,'trofeo-piva'
  ,'circuito-belvedere'
  ,'world-championships-u23'
  ,'european-championships']

rest_races = ['le-triptyque-des-monts-et-chateaux'
  ,'olympias-tour'
  ,'tour-de-normadie'
  ,'tour-de-berlin'
  ,'paris-arras-tour'
  ,'paris-tours-u23'
  ,'eschborn-frankfurt-u23'
  ,'zlm-tour']

stageraces_climb = ['ronde-de-l-isard'
  ,'tour-alsace']

stageraces_MJ = ['tour-du-valromey'
,'liege-la-gleize'
,'course-de-la-paix-junior'
,'grand-prix-ruebliland'
,'keizer-der-juniores'
,'la-coupe-du-president-de-la-ville-de-grudziadz'
,'trofeo-karlsberg'
,'sint-martinusprijs-kontich'
,'driedaagse-van-axel'
,'tour-de-l-abitibi'
,'tour-du-pays-de-vaud'
,'giro-internazionale-della-lunigiana'
,'tour-de-l-abitibi'
,'int-junioren-rundfahrt-niedersachsen']

ITT_races = [
  'uci-world-championships-itt-mj'
  ,'chrono-des-nations-les-herbiers-vendee-mj'
  ,'world-championships-itt-u23'
  ,'european-championships-itt-u23']

big_tours = ['tour-de-l-avenir'
             ,'giro-ciclistico-d-italia'
             ,'giro-ciclistico-della-valle-d-aosta-mont-blanc'
             ,'tour-des-pays-de-savoie']

In [ ]:
oneday_races_cobbles_col = [s for s in full.columns if any(xs in s for xs in oneday_races_cobbles)]
oneday_races_MJ_col = [s for s in full.columns if any(xs in s for xs in oneday_races_MJ)]
oneday_races_hilly_U23_col = [s for s in full.columns if any(xs in s for xs in oneday_races_hilly_U23)]
rest_races_col = [s for s in full.columns if any(xs in s for xs in rest_races)]
stageraces_climb_col = [s for s in full.columns if any(xs in s for xs in stageraces_climb)]
stageraces_MJ_col = [s for s in full.columns if any(xs in s for xs in stageraces_MJ)]
ITT_races_col = [s for s in full.columns if any(xs in s for xs in ITT_races)]
big_tours_col = [s for s in full.columns if any(xs in s for xs in big_tours)]

oneday_races_hilly_U23_col = [i for i in oneday_races_hilly_U23_col if "itt-u23" not in i]

t = [oneday_races_cobbles_col,oneday_races_MJ_col,oneday_races_hilly_U23_col,rest_races_col,stageraces_climb_col,stageraces_MJ_col,ITT_races_col,big_tours_col]
race_col = [item for sublist in t for item in sublist]

complete_var = np.setdiff1d(full.columns,race_col)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor

def impute(imputation_method, train, val, test, complete_var):
    
    if imputation_method=='mean':
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    elif  imputation_method=='regression':    
        imputer = IterativeImputer(estimator = RandomForestRegressor(n_estimators=500), max_iter=30, random_state=516)
    elif  imputation_method=='knn':  
        imputer = KNNImputer(n_neighbors=5, weights="uniform")    
    else:
        print('wrong imputation method')
    
    col_store = train.columns

    if imputation_method=='knn': 
        

        train_impute =  train[complete_var]
        test_impute = test[complete_var]
        val_impute = val[complete_var]
        
        for columns in t:
            total_col = [columns, complete_var]
            total_col = [item for sublist in total_col for item in sublist]

            train_tussen = train[total_col]
            test_tussen = test[total_col]
            val_tussen = val[total_col]

            imputer.fit(train_tussen)

            train_tussen = pd.DataFrame(imputer.transform(train_tussen))
            test_tussen = pd.DataFrame(imputer.transform(test_tussen))
            val_tussen = pd.DataFrame(imputer.transform(val_tussen))

            train_tussen.columns = total_col
            test_tussen.columns = total_col
            val_tussen.columns = total_col

            train_tussen = train_tussen[columns]
            test_tussen = test_tussen[columns]
            val_tussen = val_tussen[columns]

            train_impute.reset_index(drop=True, inplace=True)
            train_tussen.reset_index(drop=True, inplace=True)
            val_impute.reset_index(drop=True, inplace=True)
            val_tussen.reset_index(drop=True, inplace=True)
            test_impute.reset_index(drop=True, inplace=True)
            test_tussen.reset_index(drop=True, inplace=True)

            train_impute = pd.concat([train_impute, train_tussen], axis = 1, ignore_index = True)
            val_impute = pd.concat([val_impute, val_tussen], axis = 1, ignore_index = True)
            test_impute = pd.concat([test_impute, test_tussen], axis = 1, ignore_index = True)


    else:
        train_impute = train
        val_impute = val
        test_impute = test
        
        imputer.fit(train_impute)

        train_impute = pd.DataFrame(imputer.transform(train_impute))
        val_impute = pd.DataFrame(imputer.transform(val_impute))
        test_impute = pd.DataFrame(imputer.transform(test_impute))    

    train_impute.columns = col_store 
    val_impute.columns = col_store 
    test_impute.columns = col_store
       
    return(train_impute, val_impute, test_impute) 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

gammas = [0.5,1,1.5,2]
lrs = [0.001,0.01,0.1,0.2,0.5]
num_rounds = [2,5,10,20,50,100]
lambdas = [0.01,0.02, 0.05, 0.1, 0.2, 0.5, 1, 2]

def render_predictions(X_train, y_train, X_val, y_val, X_test):

    X_trainBig = pd.concat([X_train, X_val], axis = 0)    
    y_trainBig = np.concatenate((y_train, y_val))

    X_trainBig.columns = X_train.columns

    print('Linear Regression')

    linreg = LinearRegression()
    linreg.fit(X_trainBig, y_trainBig)    
    predictions_linreg = linreg.predict(X_test)

    print('Ridge Regression')

    waarde = 999999
    final_alpha = 0

    for lam in lambdas:
        ridge_mod = Ridge(alpha = lam)
        ridge_mod.fit(X_train, y_train)
        predictions = ridge_mod.predict(X_val)
        waarde2 = mean_squared_error(y_val, predictions, squared=False)
        if waarde2<waarde:                    
                    final_alpha = lam
                    waarde = waarde2
                    print(waarde2)
                    print('update')

    ridge_mod = Ridge(alpha = final_alpha)
    ridge_mod.fit(X_trainBig, y_trainBig)    
    predictions_ridge = ridge_mod.predict(X_test)

    print('XGBoost')
    
    waarde = 999999
    final_gamma = 0
    final_lr = 0
    final_rounds = 0

    for gamma in gammas:
        for lr in lrs:
            for rounds in num_rounds:
                xgb_model = XGBRegressor(objective="reg:squarederror", n_estimators=rounds, gamma = gamma, eta = lr)
                xgb_model.fit(X_train, y_train)
                predictions = xgb_model.predict(X_val)
                waarde2 = mean_squared_error(y_val, predictions, squared=False)
                if waarde2<waarde:                    
                    final_gamma = gamma
                    final_lr = lr
                    final_rounds = rounds
                    waarde = waarde2
                    print(waarde2)
                    print('update')
                  
    xgb_model = XGBRegressor(objective="reg:squarederror", n_estimators=rounds, gamma = gamma, eta = lr)

    xgb_model.fit(X_trainBig, y_trainBig)
    
    predictions_xgb = xgb_model.predict(X_test)

    print('Random Forest')

    rf = RandomForestRegressor(n_estimators=500)
    rf.fit(X_trainBig, y_trainBig)    
    predictions_rf = rf.predict(X_test)

    return(predictions_linreg, predictions_ridge, predictions_xgb, predictions_rf)

In [ ]:
from scipy.stats import spearmanr, percentileofscore

def validation(predictions, y_test):
  rmse = mean_squared_error(y_test, predictions, squared=False)
  spearman = spearmanr(y_test, predictions)

  #convert to percentile
  pred_percentile = np.vectorize(lambda x: percentileofscore(predictions, x))(predictions)
  true_percentile = np.vectorize(lambda x: percentileofscore(y_test, x))(y_test)

  conditions = [
    (pred_percentile<50),
    ((pred_percentile>=50) & (pred_percentile<75)),
    ((pred_percentile>=75) & (pred_percentile<90)),
    (pred_percentile>=90),
  ]
  choices = ["bottom50", "top50", "top25", "top10"]

  pred_categorized = np.select(conditions, choices)

  conditions = [
    (true_percentile<50),
    ((true_percentile>=50) & (true_percentile<75)),
    ((true_percentile>=75) & (true_percentile<90)),
    (true_percentile>=90),
  ]

  true_categorized = np.select(conditions, choices)

  accuracy = sum(pred_categorized==true_categorized)/len(true_categorized)

  total_misclass = sum((true_categorized=='bottom50')&(pred_categorized=='top10'))+sum((true_categorized=='bottom50')&(pred_categorized=='top25'))+sum((true_categorized=='top50')&(pred_categorized=='top10')) + sum((pred_categorized=='bottom50')&(true_categorized=='top10'))+sum((pred_categorized=='bottom50')&(true_categorized=='top25'))+sum((pred_categorized=='top50')&(true_categorized=='top10'))

  one_off = 1-(total_misclass/len(true_categorized))

  return([rmse, spearman, accuracy, one_off])

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def feat_sel(train, val, test, y_train):
  mi = mutual_info_regression(train, y_train)
  mi /= np.max(mi)

  selected_col = train.columns[mi>0]

  train = train[selected_col]
  val = val[selected_col]
  test = test[selected_col]

  return(train, val, test)

In [ ]:
def overall_loop(year):
  
  print(year)  
  val_year = year-1

  train = full[full['Pro_year']<val_year]
  val = full[full['Pro_year']==val_year]
  test = full[full['Pro_year']==year]

  train_rider = train['Rider']
  val_rider = val['Rider']
  test_rider = test['Rider']

  y_train = train['Points_as_neopro']
  y_val = val['Points_as_neopro']
  y_test = test['Points_as_neopro']

  train = train.drop(['Rider', 'Pro_year', 'Points_as_neopro'], axis=1)
  val = val.drop(['Rider', 'Pro_year', 'Points_as_neopro'], axis=1)
  test = test.drop(['Rider', 'Pro_year', 'Points_as_neopro'], axis=1)

  complete_var2 = list(complete_var)
  complete_var2.remove('Rider')
  complete_var2.remove('Pro_year')
  complete_var2.remove('Points_as_neopro')

  train_knn, val_knn, test_knn = impute('knn', train, val, test, complete_var2)
  train_mean, val_mean, test_mean = impute('mean', train, val, test, complete_var2)
  train_regression, val_regression, test_regression = impute('regression', train, val, test, complete_var2)

  train_knn, val_knn, test_knn = feat_sel(train_knn, val_knn, test_knn)
  train_mean, val_mean, test_mean = feat_sel( train_mean, val_mean, test_mean)
  train_regression, val_regression, test_regression = feat_sel(train_regression, val_regression, test_regression)

  print('knn')
  predictions_linreg_knn, predictions_ridge_knn, predictions_xgb_knn, predictions_rf_knn = render_predictions(train_knn,  y_train, val_knn, y_val, test_knn)
  print('mean')
  predictions_linreg_mean, predictions_ridge_mean, predictions_xgb_mean, predictions_rf_mean = render_predictions(train_mean, y_train, val_mean, y_val, test_mean)
  print('regression')
  predictions_linreg_regression, predictions_ridge_regression, predictions_xgb_regression, predictions_rf_regression = render_predictions(train_regression, y_train, val_regression, y_val, test_regression)
  
  validation_linreg_knn = validation(predictions_linreg_knn, y_test)
  validation_ridge_knn = validation(predictions_ridge_knn, y_test)
  validation_xgb_knn = validation(predictions_xgb_knn, y_test)
  validation_rf_knn = validation(predictions_rf_knn, y_test)

  validation_linreg_mean = validation(predictions_linreg_mean, y_test)
  validation_ridge_mean = validation(predictions_ridge_mean, y_test)
  validation_xgb_mean = validation(predictions_xgb_mean, y_test)
  validation_rf_mean = validation(predictions_rf_mean, y_test)

  validation_linreg_regression = validation(predictions_linreg_regression, y_test)
  validation_ridge_regression = validation(predictions_ridge_regression, y_test)
  validation_xgb_regression = validation(predictions_xgb_regression, y_test)
  validation_rf_regression = validation(predictions_rf_regression, y_test)

  return([validation_linreg_knn,validation_ridge_knn ,validation_xgb_knn ,validation_rf_knn ,validation_linreg_mean ,validation_ridge_mean ,validation_xgb_mean ,validation_rf_mean, validation_linreg_regression ,validation_ridge_regression,validation_xgb_regression ,validation_rf_regression])

In [ ]:
import random
random.seed(123)
outcome_2015 = overall_loop(2015)
outcome_2016 = overall_loop(2016)
outcome_2017 = overall_loop(2017)
outcome_2018 = overall_loop(2018)
outcome_2019 = overall_loop(2019)

2015


In [ ]:
outcome_2015

In [ ]:
outcome_2016

In [ ]:
outcome_2017

In [ ]:
outcome_2018

In [ ]:
outcome_2019